## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

## TODO

Make a version with flexible batch size.
I think the zero state is causing a problem with this right now.
I think get_loss_batch should now have flex batch size, will have to see...

In [1]:
ls evotuned/unirep/

model-13120.data-00000-of-00001  model-13120.index  model-13120.meta


In [1]:
import warnings
warnings.filterwarnings('ignore')
USE_FULL_1900_DIM_MODEL = True # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [14]:
ls ./1900_weights/

embed_matrix:0.npy             rnn_mlstm_mlstm_gmx:0.npy
fully_connected_biases:0.npy   rnn_mlstm_mlstm_gx:0.npy
fully_connected_weights:0.npy  rnn_mlstm_mlstm_wh:0.npy
rnn_mlstm_mlstm_b:0.npy        rnn_mlstm_mlstm_wmh:0.npy
rnn_mlstm_mlstm_gh:0.npy       rnn_mlstm_mlstm_wmx:0.npy
rnn_mlstm_mlstm_gmh:0.npy      rnn_mlstm_mlstm_wx:0.npy


In [2]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
from unirep import babbler64

# MODEL_WEIGHT_PATH = "./evotuned/unirep/"
MODEL_WEIGHT_PATH = "./1900_weights_random/"

In [4]:
ls ./evotuned/random_init

ls: cannot access './evotuned/random_init': No such file or directory


## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [4]:
batch_size = 100
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
dim is deprecated, use axis instead

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use tf.random.categorical instead.

In [5]:
ls evotuned/unirep/

model-13120.data-00000-of-00001  model-13120.index  model-13120.meta


In [5]:
b.load_ckpt("evotuned/unirep/model-13120")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from evotuned/unirep/model-13120


In [6]:
import pandas as pd

gfp_df = pd.read_csv('/home/david/DeepSeqGen/DB/GFP/amino_acid_seqs_brightnesses.csv')

In [7]:
gfp_df = gfp_df[~gfp_df.seq.str.contains('\*')]

In [11]:
# gfp_df['b64loss'] = gfp_df['seq'].apply(lambda x: b.get_loss(x)[0])

In [8]:
gfp2 = gfp_df[gfp_df['nMutations']==2]

In [10]:
len(gfp2)

12777

In [13]:
s0 = gfp2.iloc[0]['seq']
s1 = gfp2.iloc[1]['seq']
s0

'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRGEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQRNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

In [ ]:
log = b.get_logits(s0)

In [14]:
seqs = list(gfp2.iloc[100:500]['seq'].values)

In [15]:
l = b.get_loss_batch(seqs)

InvalidArgumentError: Input to reshape is a tensor with 2390000 values, but the requested shape has 597500
	 [[node Reshape_1 (defined at /home/david/UniRep/unirep.py:392) ]]

Caused by op 'Reshape_1', defined at:
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/david/anaconda2/envs/unirep/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, 

In [11]:
lmu = np.mean(l, axis=1)

In [12]:
l.shape

(100, 239)

In [13]:
from scipy.stats import pearsonr

pearsonr(lmu, gfp2.iloc[:100]['medianBrightness'].values)

(-0.15774348241992586, 0.11701350417677793)

In [11]:
np.mean(l[1]),l[0]

(2.9505665, array([15.228041 ,  4.196638 ,  2.3327641,  2.1869454,  2.957892 ,
         2.7660801,  2.6218424,  2.1143975,  3.0972483,  2.9134154,
         2.962264 ,  2.5209036,  2.5290055,  2.9285834,  2.7476447,
         2.205751 ,  2.4160419,  2.5031657,  2.1747038,  2.4159036,
         2.847284 ,  2.7067754,  2.2846222,  3.501367 ,  2.7754993,
         3.718196 ,  2.931644 ,  3.1107607,  2.7477875,  2.703228 ,
         2.7090495,  2.711367 ,  2.6721754,  2.6337419,  2.7019348,
         2.472194 ,  2.7263684,  2.4439402,  2.854723 ,  3.2311502,
         2.6567402,  2.9906926,  2.2066052,  3.020525 ,  2.74091  ,
         2.7993312,  3.595065 ,  3.4268396,  4.013947 ,  2.9715457,
         2.851137 ,  2.413492 ,  2.6300087,  2.4387846,  2.7904215,
         2.563156 ,  2.766084 ,  4.1388187,  2.755068 ,  2.8051677,
         2.5292563,  2.6264594,  2.8596478,  2.7056394,  2.4961562,
         2.474586 ,  3.4174933,  2.4407902,  2.5781536,  3.375768 ,
         3.7161355,  3.1241922,  2.42

In [12]:
np.mean(l, axis=1)

array([2.9655638, 2.9610941], dtype=float32)

In [11]:
av_h, h_f, c_f = b.get_rep(s)
# losses, logits = b.get_loss(s)

In [12]:
av_h.shape

(64,)

UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [4]:
seq = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [5]:
np.array(b.format_seq(seq))

array([24,  1,  2,  4, 13,  6,  6, 21, 18,  8, 13, 16, 16, 14, 17, 21, 16,
        6, 21,  5, 13,  5, 16,  9, 13,  3,  4, 18,  7, 16,  2, 13,  6, 13,
        6, 13,  5, 15,  8,  9, 13,  4, 21,  8, 21,  4, 18, 17, 11,  8,  8,
       13,  4, 21, 14, 16, 14, 20, 14,  8, 21, 16,  8,  8, 21,  8, 19, 13,
       16, 10, 11, 18, 15,  2, 19, 14,  5,  3,  1,  4, 10,  3,  5, 18, 18,
        4,  7, 15,  1, 14,  6, 13, 19, 16, 10,  6,  2,  8, 17,  7, 18,  4,
        5,  5, 13,  8, 19,  4,  8,  2, 15,  6, 16,  4, 18,  6, 13,  5,  8,
       21, 16,  9,  2, 17,  6, 21,  4, 13, 17,  5, 18,  4,  6,  5, 13,  9,
       17, 21, 13,  3,  4, 21,  6, 19,  9, 18,  9,  7,  3,  9, 16, 19, 17,
        8, 15,  5,  4, 10,  4,  9, 13, 17,  4, 15,  9, 18,  4, 17,  2,  3,
        9, 16,  6,  5, 13,  7, 16, 10, 21, 15,  5,  3, 19, 10, 10,  9,  8,
       14, 17, 13,  5, 13, 14, 16, 21, 21, 14,  5,  9,  3, 19, 21,  7,  8,
       10,  7, 16, 21,  7,  4,  5, 14,  9,  6,  4,  2,  5,  3,  1, 16, 21,
       21,  6, 18, 16,  8

We also provide a function that will check your amino acid sequences don't contain any characters which will break the UniRep model.

In [7]:
b.is_valid_seq(seq)

True

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [8]:
# Before you can train your model, 
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

This is what the integer format looks like

In [9]:
!head -n1 formatted.txt

24,1,2,4,13,6,6,21,18,8,13,16,16,14,17,21,16,6,21,5,13,5,16,9,13,3,4,18,7,16,2,13,6,13,6,13,5,15,8,9,13,4,21,8,21,4,18,17,11,8,8,13,4,21,14,16,14,20,14,8,21,16,8,8,21,8,19,13,16,10,11,18,15,2,19,14,5,3,1,4,10,3,5,18,18,4,7,15,1,14,6,13,19,16,10,6,2,8,17,7,18,4,5,5,13,8,19,4,8,2,15,6,16,4,18,6,13,5,8,21,16,9,2,17,6,21,4,13,17,5,18,4,6,5,13,9,17,21,13,3,4,21,6,19,9,18,9,7,3,9,16,19,17,8,15,5,4,10,4,9,13,17,4,15,9,18,4,17,2,3,9,16,6,5,13,7,16,10,21,15,5,3,19,10,10,9,8,14,17,13,5,13,14,16,21,21,14,5,9,3,19,21,7,8,10,7,16,21,7,4,5,14,9,6,4,2,5,3,1,16,21,21,6,18,16,8,15,15,13,17,8,3,13,1,5,6,21,19,4


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [10]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24  1  6 ... 13 18  6]
 [24  1  7 ...  0  0  0]
 [24  1  7 ...  0  0  0]
 ...
 [24  1  4 ...  0  0  0]
 [24  1 10 ...  0  0  0]
 [24  1 13 ...  0  0  0]]
(12, 265)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [12]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [13]:
y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
initializer = tf.contrib.layers.xavier_initializer(uniform=False)

with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )

loss = tf.losses.mean_squared_error(y_placeholder, prediction)

You can specifically train the top model first by isolating variables of the "top" scope, and forcing the optimizer to only optimize these.

In [14]:
learning_rate=.001
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [15]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([265, 195, 239, 176, 230, 251,  97, 223, 148, 223, 248, 205])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [16]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
                  
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 1757.400390625
Iteration 1: 1763.5225830078125
Iteration 2: 1736.2425537109375
Iteration 3: 1744.5452880859375
Iteration 4: 1726.3717041015625
Iteration 5: 1711.666015625
Iteration 6: 1719.5970458984375
Iteration 7: 1693.2880859375
Iteration 8: 1701.9228515625
Iteration 9: 1683.3359375


We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.

In [17]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        
        print("Iteration {0}: {1}".format(i,loss_))

Iteration 0: 1757.400390625
Iteration 1: 1720.1832275390625
Iteration 2: 1650.4283447265625
Iteration 3: 1552.2196044921875
Iteration 4: 1369.4599609375
Iteration 5: 1174.6700439453125
Iteration 6: 981.3558959960938
Iteration 7: 798.6965942382812
Iteration 8: 715.9775390625
Iteration 9: 580.2219848632812
